In [1]:
import numpy as np
from PMM.PMMInverse import PMMI, mode_overlap, field_mag_int, c
from ceviche import fdfd_ez
import os
import matplotlib as mpl         # These two lines just make plots render more clearly
mpl.rcParams['figure.dpi']= 500  # in the notebook

In [2]:
def Setup_Domain():
    a = 0.020
    res = 16
    nx = 18
    ny = 22
    dpml = 2
    b_o = 0.0075/a
    b_i = 0.0065/a

    ## Set up domain geometry #####################################################
    PPC = PMMI(a, res, nx, ny, dpml) #Initialize PMMI object
    PPC.Add_INFOMW_Horn(np.array([6.5, 11]), np.array([1,0]), 6.5, pol ='TM')
    PPC.Add_INFOMW_Horn(4*np.array([0.5, 3**0.5/2])+np.array([10.5,11]), np.array([-0.5,-3**0.5/2]), 8, pol ='TM')
    PPC.Add_INFOMW_Horn(4*np.array([0.5, -3**0.5/2])+np.array([10.5,11]), np.array([-0.5,3**0.5/2]), 8, pol ='TM')
    PPC.Design_Region((6.5, 6.5), (8, 9)) #Specify Region where elements are being optimized

    uniform = True
    PPC.Rod_Array_Hexagon_train(np.array([10.5,11]), 5, b_i, 1,\
                          a_basis = np.array([[0,1],[np.sqrt(3)/2,1./2]]),\
                          bulbs = True, r_bulb = (b_i, b_o), eps_bulb = 3.8,
                          uniform = uniform) #Rod ppc array


    ## Set up Sources and Sim #####################################################
    w = PPC.gamma(4.98e9) #Source frequency
    wpmax = PPC.gamma(14e9)
    gamma = 0#PPC.gamma(1e9)

    ew = 0.048/a/2-0.004/a
    hd = 0.089/a
    x = np.array([1,0])
    y = np.array([0,1])
    horn_dir_1 = np.array([0.5,3**0.5/2])
    horn_dir_2 = np.array([0.5,-3**0.5/2])
    open_dir_1 = np.array([3**0.5/2,-0.5])
    open_dir_2 = np.array([3**0.5/2,0.5])
    cen = np.array([10.5,11])

    PPC.Add_Source(np.array([6.5-hd,11-ew]), np.array([6.5-hd,11+ew]), w, 'src', 'ez')
    PPC.Add_Probe((4+hd)*horn_dir_1 + ew*open_dir_1 + cen,\
              (4+hd)*horn_dir_1 - ew*open_dir_1 + cen, w, 'prb1', 'ez')
    PPC.Add_Probe((4+hd)*horn_dir_2 + ew*open_dir_2 + cen,\
              (4+hd)*horn_dir_2 - ew*open_dir_2 + cen, w, 'prb2', 'ez')
    PPC.Add_Probe(-4.8*(3**0.5/2)*horn_dir_1 + 2.05*open_dir_1 + cen,\
              -4.8*(3**0.5/2)*horn_dir_1 - 2.05*open_dir_1 + cen, w, 'loss_ul', 'ez')
    PPC.Add_Probe(-4.8*(3**0.5/2)*horn_dir_2 + 2.05*open_dir_2 + cen,\
              -4.8*(3**0.5/2)*horn_dir_2 - 2.05*open_dir_2 + cen, w, 'loss_ll', 'ez')
    PPC.Add_Probe(4.8*(3**0.5/2)*x - 2.05*y + cen,\
              4.8*(3**0.5/2)*x + 2.05*y + cen, w, 'loss_R', 'ez')
    
    return PPC

def Run_Evaluate(PPC, rho, E0 = None, E0l = []):
    epsr_init = PPC.Rho_Parameterization_wp(rho,\
                    PPC.sources['src'][1]*PPC.a/2/np.pi/c, wp_max = PPC.gamma(14e9),\
                    gamma = 0, uniform = True)
    sim = fdfd_ez(PPC.sources['src'][1], PPC.dl, epsr_init, [PPC.Npml, PPC.Npml])
    _, _, E = sim.solve(PPC.sources['src'][0])

    if E0 == None:
        E0 = mode_overlap(E, PPC.probes['prb1'][0])
    if E0l == []:
        for prb_key in ['prb2', 'loss_ul', 'loss_ll', 'loss_R']:
            E0l.append(field_mag_int(E, PPC.probes[prb_key][3]))

    obj = mode_overlap(E, PPC.probes['prb1'][0])/E0
    i=0 
    for prb_key in ['prb2', 'loss_ul', 'loss_ll', 'loss_R']:
        obj -= 2*field_mag_int(E, PPC.probes[prb_key][3])/E0l[i]
        i+=1
    return obj, E0, E0l

def Plot_PPC(PPC, rho, fname):
    output = os.getcwd()+'/../outputs'
    PPC.Viz_Sim_abs_opt(rho, ['src'], output+fname+'.pdf',\
                    plasma = True, wp_max = PPC.gamma(14e9),\
                    uniform = True, gamma = 0)
    
def Check_Domain(PPC, rho, fname):
    output = os.getcwd()+'/../outputs'
    PPC.Viz_Domain_opt(rho, output+fname+'_domain.pdf',\
                    w = PPC.gamma(5e9), src_names = ['src'],\
                    prb_names = ['prb1', 'prb2', 'loss_ul', 'loss_ll', 'loss_R'],\
                    plasma = True, wp_max = PPC.gamma(14e9), uniform = True,\
                    gamma = 0)

In [ ]:
PPC = Setup_Domain()

rod_eps = 0.999*np.ones(61) #Rod perm values, start with plasma ~off
rho_init = PPC.Eps_to_Rho(epsr = rod_eps, plasma = True, w_src = PPC.gamma(4e9),\
                    wp_max = PPC.gamma(14e9)) #Initial Parameters

obj, E0, E0l = Run_Evaluate(PPC, rho_init)
print(obj)
#Plot_PPC(PPC, rho_init, 'Hex5Wvg_plasmaoff')

In [ ]:
rod_eps[0:11:2] = -100
on = [12,14,16,18,20,22,24,26,28,30,32,34,36,38,40,50,52]
rod_eps[on] = -100
rho_plasma = PPC.Eps_to_Rho(epsr = rod_eps, plasma = True, w_src = PPC.gamma(4e9),\
                    wp_max = PPC.gamma(14e9))

#obj, E0, E0l = Run_Evaluate(PPC, rho_plasma, E0, E0l)
#print(obj)
PPC.Clear_fields()
Plot_PPC(PPC, rho_plasma, 'Hex5Wvg_plasmaon')
Check_Domain(PPC, rho_plasma, 'Hex5Wvg_plasmaon')